# Bigquery

> Everything needed to get started to query from and query to 

In [ ]:
#| default_exp bigquery

In [ ]:
#| hide
from nbdev.showdoc import *
from google.cloud import bigquery

## Initilize Client

In [2]:
def create_client(
    auth_scopes: [str] = [],  # list of auth scopes eg: ["bigquery", "drive"]
):
    """Create a BigQuery client with the given auth scopes."""
    if len(auth_scopes) == 0:
        return bigquery.Client()
    else:
        import google.auth

        credentials, project = google.auth.default(
            [f"https://www.googleapis.com/auth/{service}" for service in []]
        )
        return bigquery.Client(
            credentials=credentials,
            project=project,
        )


In [ ]:
#| export
def query_to_dataframe(bq_client, sql):
    """
    Function to execute the sql query
    Args:
        bq_client: Bigquery client
        sql : sql query
    Returns:
        Output query Results as a Data frame
    """
    dataframe = bq_client.query(sql).to_dataframe()

    return dataframe

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()